In [94]:
import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', None)
pd.plotting.register_matplotlib_converters()
import pygwalker as pyg

import numpy as np
import seaborn as sns
import missingno as mgn

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt

pio.templates.default = "simple_white"

In [16]:
df = pd.read_csv('/Users/clara/Desktop/neuefische/d-drivers/data/data_features.csv')
#df.drop([])

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6815 entries, 0 to 6814
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   page_id                 6815 non-null   int64  
 1   n_days                  6815 non-null   int64  
 2   url                     6815 non-null   object 
 3   no_versions             6815 non-null   int64  
 4   last_publish_date       6815 non-null   object 
 5   word_count              6815 non-null   float64
 6   classification_product  6815 non-null   object 
 7   classification_type     6815 non-null   object 
 8   page_name               6815 non-null   object 
 9   title                   6815 non-null   object 
 10  last_author             6815 non-null   object 
 11  external_clicks         6815 non-null   float64
 12  external_impressions    6815 non-null   float64
 13  likes_n_days            6815 non-null   float64
 14  total_likes_n_days      6815 non-null   

In [18]:
# Summarize all target variables in a list
target = ['crt', 'external_clicks', 'external_impressions']

# Summarize all numeric variables in a list
num_variables = df.select_dtypes(include=[np.number]).columns.tolist()
# columns_to_remove = ['external_clicks', 'external_impressions', 'ctr']
# for col in columns_to_remove:
#     num_variables.remove(col)

In [19]:
# Compute the correlation matrix
corr_matrix = df[num_variables].corr()

# Print the correlation matrix
print(corr_matrix)

# Create heatmap of correlation matrix
fig = px.imshow(corr_matrix, color_continuous_scale='RdBu', title='Correlation Matrix')
fig.show()

                       page_id    n_days  no_versions  word_count   
page_id               1.000000 -0.300763    -0.167048   -0.086368  \
n_days               -0.300763  1.000000     0.624697    0.162702   
no_versions          -0.167048  0.624697     1.000000    0.195596   
word_count           -0.086368  0.162702     0.195596    1.000000   
external_clicks       0.002478  0.238998     0.289466    0.035535   
external_impressions  0.001997  0.267615     0.324785    0.044973   
likes_n_days         -0.046476  0.104402     0.104603    0.055622   
total_likes_n_days   -0.039736  0.054709     0.064154    0.039406   
video_play            0.006177  0.276891     0.310121    0.047537   
page_impressions      0.009198  0.309372     0.337487    0.057033   
clickouts            -0.058957  0.473494     0.553420    0.156360   
ctr                  -0.188644  0.080662    -0.022485   -0.163384   
scraped_word_count   -0.382895  0.317928     0.249505    0.631193   
meta_title_len        0.128259 -0.

In [20]:
# Compute the correlation matrix
corr_matrix = df[num_variables].corr()

# Set the threshold for significant correlations
threshold = 0.5  # You can adjust this threshold as needed

# Create a mask to filter out insignificant correlations
mask = np.abs(corr_matrix) >= threshold
corr_matrix_filtered = corr_matrix[mask]

# Print the filtered correlation matrix
print(corr_matrix_filtered)

# Create heatmap of filtered correlation matrix
fig = px.imshow(corr_matrix_filtered, color_continuous_scale='RdBu', title='Significant Correlations (absolute value >= 0.5)')
fig.show()

                      page_id    n_days  no_versions  word_count   
page_id                   1.0       NaN          NaN         NaN  \
n_days                    NaN  1.000000     0.624697         NaN   
no_versions               NaN  0.624697     1.000000         NaN   
word_count                NaN       NaN          NaN    1.000000   
external_clicks           NaN       NaN          NaN         NaN   
external_impressions      NaN       NaN          NaN         NaN   
likes_n_days              NaN       NaN          NaN         NaN   
total_likes_n_days        NaN       NaN          NaN         NaN   
video_play                NaN       NaN          NaN         NaN   
page_impressions          NaN       NaN          NaN         NaN   
clickouts                 NaN       NaN     0.553420         NaN   
ctr                       NaN       NaN          NaN         NaN   
scraped_word_count        NaN       NaN          NaN    0.631193   
meta_title_len            NaN       NaN         

In [21]:
walker = pyg.walk(df)

Box(children=(HTML(value='<div id="ifr-pyg-000615bdd8b5c167WnsvAL0yEpZta5Bq" style="height: auto">\n    <head>…

In [191]:
# Import linear regression from sklearn library
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, confusion_matrix, accuracy_score, classification_report,mean_absolute_error, mean_squared_error

In [192]:
df = pd.read_csv('/Users/clara/Desktop/neuefische/d-drivers/data/data_features.csv')

In [193]:
df.dropna(axis=0, how='any', inplace=True)
df.drop(['page_id','url','video_play', 'page_impressions', 'clickouts','last_author','date_scraped','scraped_word_count','meta_title', 'meta_description',
         'abstract','meta_image_url','page_img_size', 'merged_url','last_publish_date', 'page_name', 'title','h1'], axis=1, inplace= True)

In [194]:
df.columns

Index(['n_days', 'no_versions', 'word_count', 'classification_product',
       'classification_type', 'external_clicks', 'external_impressions',
       'likes_n_days', 'total_likes_n_days', 'ctr', 'scraped_author',
       'media_type', 'meta_title_len', 'meta_desc_len', 'h1_len',
       'abstract_len', 'merged_url_len'],
      dtype='object')

In [195]:
# Filter columns of a specific data type (e.g., numerical columns)
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()

# Filter columns of a specific data type (e.g., categorical columns)
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Filter columns of a specific data type (e.g., datetime columns)
datetime_columns = df.select_dtypes(include=['datetime']).columns.tolist()

# Combine numerical and categorical features
X_all = df[numeric_columns + categorical_columns + datetime_columns]
X_all.drop(['external_clicks','external_impressions','ctr'],axis=1,inplace=True)

# One-hot encode all categorical features
X_all_encoded = pd.get_dummies(X_all, columns=categorical_columns, drop_first=True)

/var/folders/5s/w7hst37d0tv0r4hby2_mqqp00000gn/T/ipykernel_20653/1569258532.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [196]:
X_all_encoded

,n_days,no_versions,word_count,likes_n_days,total_likes_n_days,meta_title_len,meta_desc_len,h1_len,abstract_len,merged_url_len,...,scraped_author_Yello Solar,scraped_author_dpa,scraped_author_dpa/CHIP,scraped_author_press-inform,scraped_author_press-inform/Wolfgang Gomoll,scraped_author_pv magazine,scraped_author_pv-magazine,scraped_author_smarter fahren,scraped_author_t3n,scraped_author_zolar
0,6,0,827.000000,2.0,5.0,55,64,61,217.0,2,...,False,False,False,False,False,False,False,False,False,False
1,1,0,1066.000000,0.0,0.0,46,141,75,486.0,11,...,False,False,False,False,False,False,False,False,False,False
2,10,0,466.000000,4.0,0.0,53,155,53,264.0,10,...,False,False,False,False,False,False,False,False,False,False
3,3,0,0.000000,0.0,0.0,71,156,71,314.0,12,...,False,False,False,False,False,False,False,False,False,False
5,11,1,477.454545,3.0,13.0,63,153,63,173.0,10,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6808,2,0,0.000000,0.0,0.0,62,78,62,254.0,9,...,False,False,False,False,False,False,False,False,False,False
6809,2,0,0.000000,0.0,0.0,62,71,62,172.0,8,...,False,False,False,False,False,False,False,False,False,False
6811,1,0,0.000000,0.0,0.0,77,81,77,278.0,10,...,False,False,False,False,False,False,False,False,False,False
6812,1,0,0.000000,0.0,0.0,76,153,76,383.0,11,...,False,False,False,False,False,False,False,False,False,False


In [199]:
def lin_reg_evaluation(X, y, test_size=0.3, random_state=25):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Fit the linear regression model
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)

    # Predictions on training and testing sets
    y_pred_train = lin_reg.predict(X_train)
    y_pred_test = lin_reg.predict(X_test)


    # Evaluation metrics
    print("R-squared (Train):", r2_score(y_train, y_pred_train).round(3))
    print("R-squared (Test):", r2_score(y_test, y_pred_test).round(3))
    print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred_test).round(3))
    print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred_test).round(3))
    print("Root Mean Squared Error (RMSE):", np.sqrt(mean_squared_error(y_test, y_pred_test)).round(3))


In [210]:
target = ['external_impressions', 'external_clicks', 'ctr']
columns = ['no_versions','n_days','classification_product','classification_type','scraped_author',
           'likes_n_days','total_likes_n_days','scraped_word_count','media_type','meta_title_len','meta_desc_len','h1_len','abstract_len']

X = [df[['word_count','meta_title_len', 'meta_desc_len', 'h1_len', 'abstract_len']],
    X_all_encoded,
    #df[['media_type','scraped_author','classification_product','n_days']]
    ]

for features in X:
    print(f"Evaluating for features: {features.columns}")
    for item in target:
        y = df[item]
        print(f"Evaluating for target: {item}")
        lin_reg_evaluation(features, y)
        print()

Evaluating for features: Index(['word_count', 'meta_title_len', 'meta_desc_len', 'h1_len',
       'abstract_len'],
      dtype='object')
Evaluating for target: external_impressions
R-squared (Train): 0.02
R-squared (Test): 0.031
Mean Absolute Error (MAE): 175121.075
Mean Squared Error (MSE): 137918101329.924
Root Mean Squared Error (RMSE): 371373.264

Evaluating for target: external_clicks
R-squared (Train): 0.017
R-squared (Test): 0.024
Mean Absolute Error (MAE): 13895.248
Mean Squared Error (MSE): 832356819.72
Root Mean Squared Error (RMSE): 28850.595

Evaluating for target: ctr
R-squared (Train): 0.036
R-squared (Test): 0.033
Mean Absolute Error (MAE): 2.845
Mean Squared Error (MSE): 16.062
Root Mean Squared Error (RMSE): 4.008

Evaluating for features: Index(['n_days', 'no_versions', 'word_count', 'likes_n_days',
       'total_likes_n_days', 'meta_title_len', 'meta_desc_len', 'h1_len',
       'abstract_len', 'merged_url_len',
       ...
       'scraped_author_Yello Solar', 'scraped

In [209]:
target = ['external_impressions', 'external_clicks', 'ctr']

X = X_all_encoded[['media_type','scraped_author','classification_product','n_days']]

for item in target:
    y = df[item]
    print(f"Evaluating for target: {item}")
    lin_reg_evaluation(X, y)
    print()

KeyError: "['media_type', 'scraped_author', 'classification_product'] not in index"